In [7]:
#import mobi
#import tempfile
#import shutil
#from extract import extract
#tempdir, filepaths = extract(r"C:\Users\Mateusz\Downloads\Steinbeck John - Grona gniewu.mobi")
#print(filepaths)
#shutil.rmtree(tempdir)

In [63]:
import openmeteo_requests
import requests_cache
from retry_requests import retry

from datetime import datetime
from opensearchpy import OpenSearch, helpers
from dateutil.relativedelta import relativedelta

In [64]:
""" df = pd.read_csv("zadanie3_projekt/app/europe_cities.csv") """

' df = pd.read_csv("zadanie3_projekt/app/europe_cities.csv") '

In [65]:
#df = df[(df['lat']>36) & (df['lng']>-10) & (df['lng'] < 50) & (df['population'] > 350000 ) ]
#df = df.sort_values("id")
#df.to_csv("europe_cities.csv")

In [66]:
""" lats = df["lat"].to_list()
lngs = df['lng'].to_list()
ids = df.loc[:, ['id','city','country','lat', 'lng']].values.tolist() """

' lats = df["lat"].to_list()\nlngs = df[\'lng\'].to_list()\nids = df.loc[:, [\'id\',\'city\',\'country\',\'lat\', \'lng\']].values.tolist() '

In [67]:
host = 'localhost'
port = 9200

# Create the client with SSL/TLS and hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    http_auth=('admin', 'QWERTYadmin123!@#'),  # login/admin password
)

In [69]:
query = {
    "size": 500,
  "query": {
    "match_all": {}
  }

}

resp = client.search(index="python-cities-index", body=query)
city_doc = resp["hits"]["hits"]

In [71]:
ids = [ {"id":city['_source']['id'] ,"city":city['_source']['city'] ,"country":city['_source']['country'] , "location": city['_source']['location'] } for city in city_doc]
lats = [lat['location'][1] for lat in ids]
lngs = [lng['location'][0] for lng in ids]

In [73]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": lats,
	"longitude": lngs,
	"current": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m", "cloud_cover", "surface_pressure"],
	"timezone": "Europe/Berlin",
    
}
responses = openmeteo.weather_api(url, params=params)

In [74]:
temp = dict()
result = list()

In [75]:
for id,response,lat,lng in zip(ids,responses,lats, lngs):

    current = response.Current()
    temp = dict()
    temp_datetime_id = int(datetime.fromtimestamp(current.Time()).strftime('%Y%m%d%H%M%S'))
    temp["city"] = id['city']
    temp["country"] = id['country']
    temp['location'] = id['location']
    temp["datetime_id"] = temp_datetime_id
    temp["temperature"] = current.Variables(0).Value()
    temp["humidity"] = current.Variables(1).Value()
    temp["wind_speed"] =  current.Variables(2).Value()
    temp["wind_direction"] = current.Variables(3).Value()
    temp["wind_gusts"] = current.Variables(4).Value()
    temp["cloud_cover"] = current.Variables(5).Value()
    temp["surface_pressure"] = current.Variables(6).Value()
    result.append( (f"{str(id['id'])}_{str(temp_datetime_id)}", temp) ) #tuple(id, dict)


In [76]:
""" for id,lat,lng in zip(ids,lats, lngs):

    temp = dict()
    temp["id"] = id[0]
    temp["city"] = id[1]
    temp["country"] = id[2]
    temp['location'] = [lng, lat]
    result.append( ( str(id[0]) , temp) ) #tuple(id, dict)
 """

' for id,lat,lng in zip(ids,lats, lngs):\n\n    temp = dict()\n    temp["id"] = id[0]\n    temp["city"] = id[1]\n    temp["country"] = id[2]\n    temp[\'location\'] = [lng, lat]\n    result.append( ( str(id[0]) , temp) ) #tuple(id, dict)\n '

In [78]:
def detect_weather_anomaly(record: dict) -> dict:
    anomalies = []

    temp = record["temperature"]
    wind = record["wind_speed"]
    gusts = record["wind_gusts"]
    pressure = record["surface_pressure"]
    humidity = record["humidity"]
    clouds = record["cloud_cover"]

    # Temperatura
    if temp > 35:
        anomalies.append("HIGH_TEMPERATURE")
    elif temp < -25:
        anomalies.append("LOW_TEMPERATURE")

    # Wiatr
    if wind > 20:
        anomalies.append("STRONG_WIND")
    if gusts > 30:
        anomalies.append("STRONG_WIND_GUSTS")

    # Ciśnienie
    if pressure < 980:
        anomalies.append("LOW_PRESSURE")
    elif pressure > 1040:
        anomalies.append("HIGH_PRESSURE")

    # Spójność danych
    if clouds == 0 and humidity > 90:
        anomalies.append("INCONSISTENT_CLOUD_HUMIDITY")

    # Status końcowy
    record["anomaly"] = len(anomalies) > 0
    record["anomaly_types"] = anomalies

    return record


In [80]:
result = [ (res[0],detect_weather_anomaly(res[1])) for res in result]

In [81]:
for res in result:
    if res[1]['city'] == 'Łódź':
        print(res[1])

{'city': 'Łódź', 'country': 'Poland', 'location': [19.4547, 51.7769], 'datetime_id': 20251221154500, 'temperature': 2.4000000953674316, 'humidity': 85.0, 'wind_speed': 6.839999675750732, 'wind_direction': 98.0, 'wind_gusts': 15.480000495910645, 'cloud_cover': 100.0, 'surface_pressure': 995.4260864257812, 'anomaly': False, 'anomaly_types': []}


In [42]:
# Create an index with non-default settings.
index_name = 'python-cities-index'
mapping = {
  "mappings": {
    "properties": {
      "id":          {"type": "float"}, 
      "city":        { "type": "keyword" },
      "country":     { "type": "keyword" },
      "location":    { "type": "geo_point" },
      "active":      { "type": "boolean" }
    }
  }
}

response = client.indices.create(index=index_name, body=mapping)
print('\nCreating index:')
print(response)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-cities-index'}


In [ ]:
""" actions = [
    {
        '_op_type': 'update',
        '_index': 'python-cities-index',
        '_id': res[0],                    # document id
        'doc': res[1],
        'doc_as_upsert': True          # if doc doesn't exist -> create
    } for res in result
] """

In [ ]:
""" success, failed = helpers.bulk(client, actions, refresh=True)
print(f"Success: {success}, Failed: {failed}") """

Success: 224, Failed: []


In [40]:
response = client.indices.delete(
    index = 'python-cities-index'
)

In [82]:
response = client.indices.delete(
    index = 'python-weather5-index'
)

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [python-weather5-index]', python-weather5-index, index_or_alias)

In [83]:
# Create an index with non-default settings.
index_name = 'python-weather5-index'
mapping = {
    "mappings": {
        "properties": {
            "city": {"type": "keyword"},
            "country": {"type": "keyword"},
            "location": {"type": "geo_point"},  # lat/lon for maps
            "datetime_id": {"type": "date", "format": "yyyyMMddHHmmss"},
            "temperature": {"type": "float"},
            "humidity": {"type": "float"},
            "wind_speed": {"type": "float"},
            "wind_direction": {"type": "float"},
            "wind_gusts": {"type": "float"},
            "cloud_cover": {"type": "float"},
            "surface_pressure": {"type": "float"},
            "anomaly": {
                "type": "boolean"
            },
            "anomaly_types": {
                "type": "keyword"
            },
        }
    }
}
response = client.indices.create(index=index_name, body=mapping)
print('\nCreating index:')
print(response)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-weather5-index'}


In [84]:
actions = [
    {
        '_op_type': 'update',
        '_index': 'python-weather5-index',
        '_id': res[0],                    # document id
        'doc': res[1],
        'doc_as_upsert': True          # if doc doesn't exist -> create
    } for res in result
]

In [85]:
success, failed = helpers.bulk(client, actions, refresh=True)
print(f"Success: {success}, Failed: {failed}")

Success: 224, Failed: []
